In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####

from crud import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# credentials for the databases
username = "aacuser"
password = "SNHU1234"
host = "nv-desktop-services.apporto.com"
port = 32212

# connect to database
db = AnimalShelter(username, password, host, port)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

#code for interactive dashboard
app.layout = html.Div([
    html.Center(html.B(html.H1('Dashboard'))),
    html.Center(html.Img(src=f"data:image/png;base64,{encoded_image}", style={'height': '100px'})),
    html.Center(html.B("Alex Delarosa Was here")), #unique identifier
    html.Hr(),
    #creates radios buttons for filtering purposes based on rescue type.
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
            {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
            {'label': 'Reset', 'value': 'reset'}
        ],
        value='reset', #resets to original state
        labelStyle={'display': 'inline-block'} # creates inline display for radio buttons
    ),
    html.Hr(),
    #creates data table using fetched data
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'), #formats the data into rows of data
        row_selectable='single', #allows single row selection
        page_size=10,# limits number of rows displayed by 10
        sort_action='native',# allows user to sort rows in ascending or descending order in the collumn
        filter_action='native',#allows user to filter data based on value entered in the column
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'), #place holder for chart
        html.Div(id='map-id', className='col s12 m6') #place holder for map
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback(Output('datatable-id', 'data'), [Input('filter-type', 'value')])
# define queries based on selectec filter type.
def update_dashboard(filter_type):
    if filter_type == 'water':
        query = {
            "breed": {"$in": ["Labrador Retriever", "Chesapeake Bay Retriever", "Newfoundland"]},
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'mountain':
        query = {
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'disaster':
        query = {
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
    else:
        query = {}

    df_filtered = pd.DataFrame.from_records(db.read(query))
    df_filtered.drop(columns=['_id'], inplace=True) 

    # returns filtered data for the DataTable
    return df_filtered.to_dict('records')
        
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])       
def update_graphs(viewData):
    #checks if data is available
    if viewData is None or len(viewData) == 0:
        return html.Div("Data not available")

    dff = pd.DataFrame.from_dict(viewData)
    
    # creates a pie chart
    pie_chart = px.pie(
        dff,
        names='breed',  # dog breed columnn
        title='Dog Breeds'  # title
    )
    
    # returns pie chart
    return dcc.Graph(
        figure=pie_chart
    )


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
#FIXME Add in the code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    
    if index is None or len(index) == 0:
        row = 0  
    else:
        row = index[0] 
        
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[30.75, -97.48], zoom=10, children=[
                   dl.TileLayer(id="base-layer-id"),
                   dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                              children=[
                                  dl.Tooltip(dff.iloc[row, 4]),  # animal breed
                                  dl.Popup([
                                      html.H1("Animal Name"),
                                      html.P(dff.iloc[row, 9])  # animal name
                                  ])
                              ])
               ])
    ]


app.run_server(debug=True)


Dash app running on http://127.0.0.1:18760/
